In [28]:
import csv
from qiskit import transpile
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_distribution
from qiskit_aer import AerSimulator
import qiskit.quantum_info as qi
import numpy as np
from quantum_check import construct_circuit, check_consistency
import matplotlib.pyplot as plt

In [26]:
#Load in data from data.csv
test_data = []

with open("data/data.csv", mode="r", newline="") as file: 
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        n = int(row[0])
        system_type = row[1]
        F1, F2, F3 = row[2], row[3], row[4]
        test_data.append((n, system_type, F1, F2, F3))

In [ ]:
exp = 0
terms = []
func = 0
var = 0
label = ''

for data in test_data:
    exp += 1
    var = data[0]
    label = data[1]
    for i in range(len(data) - 2):
        terms.append(data[i+2])
    func = len(terms)

    print(f"Experiment {exp}\nType: {label}\nTerms: {terms} and number of variables: {var}\n")
    
    circ = construct_circuit(terms, var, func)
    circ.draw("mpl");
    plt.savefig(f"theoretical_figures/circuit_{exp}_{terms}")
    plt.close()

    simulator = AerSimulator(method="statevector")
    basis = ['u1', 'u2', 'u3', 'cx', 'id', 'x', 'y', 'z', 'h', 's', 'sdg', 't', 'tdg', 'measure']
    circ = transpile(circ, basis_gates=basis, optimization_level=0)

    
    shots = [64, 128, 256]
    seed = 12
    
    for shot in shots:
        result = simulator.run(circ, shots=shot, seed_simulator=seed).result()
        probabilities = {k:v/shot for k,v in result.get_counts().items()}
        plot_distribution(probabilities);
        plt.savefig(f"theoretical_figures/experiemnt_{exp}_{shot}")
        plt.close()

        p0011 = probabilities.get('0011', 0)
        p1100 = probabilities.get('1100', 0)

        print(f"Shots = {shot}, {check_consistency([p0011, p1100])}")
        
    print("\n");
    
    terms = []


Experiment 1
Type: Consistent
Terms: ['0x1x', '0xx0', 'x01x'] and number of variables: 4

The system is consistent. C=0.4330127018922193
The system is consistent. C=0.375
The system is consistent. C=0.3644344934278313


Experiment 2
Type: Consistent
Terms: ['0x0xx1xx', 'xx0xxx1x', 'x10xx1xx'] and number of variables: 8

The system is consistent. C=0.39528470752104744
The system is consistent. C=0.2795084971874737
The system is consistent. C=0.2651650429449553


Experiment 3
Type: Consistent
Terms: ['xx0xx0xxxxxx', '11xxxxxxx1xx', 'x1xxx0xxx1xx'] and number of variables: 12

The system is consistent. C=0.39528470752104744
The system is consistent. C=0.2795084971874737
The system is consistent. C=0.2651650429449553


Experiment 4
Type: Inconsistent
Terms: ['0x11', '0xx0', 'x00x'] and number of variables: 4

The system is consistent. C=0.25
The system is consistent. C=0.21650635094610965
The system is consistent. C=0.19764235376052372


Experiment 5
Type: Inconsistent
Terms: ['0xxxx0xx', 